<h1><strong> 이미지 추론 및 보정 </strong></h1>
<br>

<hr>

<br>
<h2> 1. Introduction </h2>
 <p style="font-size:16px">개요 : 본 노트북은 사전 학습된 XGBoost 모델을 로드하여, 입력 이미지의 패치별 시각적 특성(평균, 표준편차, FFT 기반 고주파 비율)을 추출하고, 이를 기반으로 지역적으로 최적화된 샤프닝 강도를 추론하여 적응형 언샤프 마스킹(Adaptive Unsharp Masking)을 수행하는 과정을 기록한 리포트이다.
 </p>
 <p style="font-size:16px">목표 : 이미지의 로컬 패치 특징을 입력으로 받아 k맵을 생성하고, 이를 가중치로 사용하여 이미지 전역에 일률적인 강도가 아닌 픽셀별 최적화된 샤프닝을 적용함으로써 디테일이 향상된 결과 이미지를 생성 및 저장한다.</p>

<br>
<hr>
<br>
<h2> 2. Library Import</h2>


In [ ]:
import os
import cv2
import joblib
import numpy as np
import pandas as pd

import sys
sys.path.append("../src") 
import params as pr
import preprocessing as pre 

MODEL_NAME = "xgb_model.joblib"

<hr>
<br>
<h2>3. k맵 생성 및 추론 함수</h2>

In [ ]:
def _fft_high_freq_ratio(gray: np.ndarray, cutoff_ratio: float = 0.25) -> float:
    f = np.fft.fft2(gray.astype(np.float32))
    mag = np.abs(np.fft.fftshift(f))
    h, w = gray.shape
    cy, cx = h // 2, w // 2
    r = int(min(h, w) * cutoff_ratio)
    Y, X = np.ogrid[:h, :w]
    mask_low = (X - cx) ** 2 + (Y - cy) ** 2 <= r ** 2
    low_e = mag[mask_low].sum()
    high_e = mag[~mask_low].sum()
    return float(high_e / (low_e + high_e + 1e-8))

<p style="font-size:16px">
위 함수는 이미지 패치를 주파수 도메인으로 변환하여, 해당 패치의 디테일들을 수치화한다.

주파수 영역으로 변환하면 저주파 부분이 모서리에 모여있기 때문에, 쉽게 분석할 수 있도록 저주파 성분을 <code>np.fft.fftshift</code>로 정중앙으로 이동시키도록 했다.

이미지의 크기의 cutoff_ratio배 만큼의 원을 생성하고, 저주파 영역인 원 안의 강도(magnitude)와 고주파 영역인 원 밖의 강도(magnitude)를 구해 에너지 비율을 반환한다.

적당한 고주파 비율을 추출하기 위해 원의 크기는 25%를 기본값으로 지정했다.
</p>

In [ ]:
def dispatcher_extract_features(img_bgr: np.ndarray) -> dict:
    luma = pre.rgb_to_luma(pre.bgr_to_rgb(img_bgr))
    return pre.extract_features(luma) 

<p style="font-size:16px">
위 함수는 특징 추출 함수를 실행하는 함수이다.

rgb로 바꾼 이미지의 밝기 값을 매개변수로 추출 함수를 호출한다.
</p>

In [ ]:
def build_k_map(img_bgr: np.ndarray, model, patch=64, stride=None, k_clip=(0.0, 5.0)):
    if stride is None:
        stride = patch
    h, w = img_bgr.shape[:2]
    kh = (h + stride - 1) // stride
    kw = (w + stride - 1) // stride
    k_grid = np.zeros((kh, kw), dtype=np.float32)

    rows = []
    locs = []
    for gy, y in enumerate(range(0, h, stride)):
        for gx, x in enumerate(range(0, w, stride)):
            patch_img = img_bgr[y:min(y+patch, h), x:min(x+patch, w)]
            feats = dispatcher_extract_features(patch_img)
            rows.append([feats["mean"], feats["std"], feats["high_freq_ratio"]])
            locs.append((gy, gx))

    df = pd.DataFrame(rows, columns=["mean", "std", "high_freq_ratio"])
    k_pred = model.predict(df).astype(np.float32)
    k_pred = np.clip(k_pred, k_clip[0], k_clip[1])

    for (gy, gx), k in zip(locs, k_pred):
        k_grid[gy, gx] = k
    k_map = cv2.resize(k_grid, (w, h), interpolation=cv2.INTER_CUBIC)
    return k_map

<p style="font-size:16px">
위 함수는 입력 이미지를 stride 크기의 윈도우로 슬라이딩하며 순회하고, 각 영역의 특징을 추출하는 함수이다.

각 영역의 특징들을 격차 형태로 stride 간격 만큼 append 한다.

그 다음 모델을 호출하여 해당 영역에 대한 특징을 학습하여 샤프닝 강도인 k값들을 일괄적(batch)으로 예측하게 했다.
샤프닝 강도는 0.0에서 5.0 값으로 제한했고, 예측한 k값들을 다시 2차원 배열에 매핑을 시키게 했다.

예측된 k맵은 1.0-1.0-2.0 처럼 극단적으로 비연속적이기 때문에, <b>3차 회선 보간법(Bicubic Interpolation)</b>을 적용하여 원본 이미지 해상도 크기의 부드러운 연속적인 k맵으로 변환하도록 했다.
</p>

In [ ]:
def unsharp_with_kmap(img_bgr: np.ndarray, k_map: np.ndarray, sigma=2.0, kernel_size=0):
    img = img_bgr.astype(np.float32)
    if kernel_size and kernel_size % 2 == 1:
        blur = cv2.GaussianBlur(img, (kernel_size, kernel_size), sigma, sigma)
    else:
        blur = cv2.GaussianBlur(img, (0, 0), sigma, sigma)
    mask = img - blur
    out = img + (k_map[..., None]) * mask
    return np.clip(out, 0, 255).astype(np.uint8)

<p style="font-size:16px">
위 함수는 만들어진 k맵을 활용하여 실질적으로 이미지 보정을 수행하는 함수이다.

언샤프 마스킹 공식은 다음과 같다.
$$Output(x,y) = Image(x,y) + k\_map(x,y) \times (Image(x,y) - Blur(x,y))$$

원본이미지에서 블러이미지를 뺀 고주파 영역만 추출하고 k배한 값을 원본이미지에 더해서 출력물을 구성한다.
</p>

In [ ]:
def enhance_image_patchwise(input_path, patch, stride):
    img = cv2.imread(input_path, cv2.IMREAD_COLOR)
    sigma = pre.get_sigma(img)
    if img is None:
        raise FileNotFoundError(f"이미지 없음.")

    model_path = os.path.join(pr.output_dir, MODEL_NAME)
    model = joblib.load(model_path)
    
    k_map = build_k_map(img, model, patch, stride, k_clip=(0, 100))
    k_map_amplified = k_map * 125.0
    k_map_final = np.clip(k_map_amplified, 0.0, 5.0)

    out = unsharp_with_kmap(img, k_map_final, sigma)
    
    result_path = "./result/sharpened_img.png"
    cv2.imwrite(result_path, out)
    
    return result_path

위 함수는 전체 프로세스를 실행하는 함수이다.

이미지를 불러오고, <code>get_sigma</code> 함수로 해상도에 따른 최적의 블러 반경을 먼저 계산한다.

그 다음, 저장된 XGBoost 모델을 불러오고 <code>build_k_map</code> 함수를 호출하여 k맵 지도를 생성한다.
k_map의 샤프닝 강도인 k값은 매우 작은 수치이기 때문에 증폭을 시켜야 하는데, 샘플 이미지의 최대 샤프닝 강도가 3.0이 적정하다는 근거 하에 125배 증폭시킨다.




<hr>
<br>
<h2> 4. 결론 </h2>
<p style="font-size:16px">
- 사전 학습된 XGBoost 모델을 로드하여, 입력 이미지의 패치별 통계적 특성(Mean, Std) 및 고주파 비율을 기반으로 지역별 최적의 샤프닝 강도 k를 추론하였다.<br>
- 평가지표로 RMSE, RMAE, R2로 과적합, 과격한 오차, 학습률을 비교했다.<br>
    <br><br>

</p>